# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc9401182e0>
├── 'a' --> tensor([[ 1.4557, -0.8750,  0.9974],
│                   [-0.4267, -2.5494, -0.8116]])
└── 'x' --> <FastTreeValue 0x7fc940118700>
    └── 'c' --> tensor([[ 1.5550,  1.7717,  0.8875, -0.2871],
                        [-0.0854, -0.6550,  0.6069, -0.3160],
                        [-0.1392,  1.0513,  1.9036, -1.3792]])

In [4]:
t.a

tensor([[ 1.4557, -0.8750,  0.9974],
        [-0.4267, -2.5494, -0.8116]])

In [5]:
%timeit t.a

63.8 ns ± 0.061 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc9401182e0>
├── 'a' --> tensor([[ 0.0604,  0.6563,  2.0124],
│                   [-0.0947,  0.0063, -0.3051]])
└── 'x' --> <FastTreeValue 0x7fc940118700>
    └── 'c' --> tensor([[ 1.5550,  1.7717,  0.8875, -0.2871],
                        [-0.0854, -0.6550,  0.6069, -0.3160],
                        [-0.1392,  1.0513,  1.9036, -1.3792]])

In [7]:
%timeit t.a = new_value

70 ns ± 0.119 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.4557, -0.8750,  0.9974],
               [-0.4267, -2.5494, -0.8116]]),
    x: Batch(
           c: tensor([[ 1.5550,  1.7717,  0.8875, -0.2871],
                      [-0.0854, -0.6550,  0.6069, -0.3160],
                      [-0.1392,  1.0513,  1.9036, -1.3792]]),
       ),
)

In [10]:
b.a

tensor([[ 1.4557, -0.8750,  0.9974],
        [-0.4267, -2.5494, -0.8116]])

In [11]:
%timeit b.a

57.8 ns ± 0.139 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0845,  0.1974,  0.4168],
               [ 0.9878,  1.6085,  0.2184]]),
    x: Batch(
           c: tensor([[ 1.5550,  1.7717,  0.8875, -0.2871],
                      [-0.0854, -0.6550,  0.6069, -0.3160],
                      [-0.1392,  1.0513,  1.9036, -1.3792]]),
       ),
)

In [13]:
%timeit b.a = new_value

508 ns ± 0.144 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

809 ns ± 1.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 8.12 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 3.73 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 293 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc894e02f40>
├── 'a' --> tensor([[[ 1.4557, -0.8750,  0.9974],
│                    [-0.4267, -2.5494, -0.8116]],
│           
│                   [[ 1.4557, -0.8750,  0.9974],
│                    [-0.4267, -2.5494, -0.8116]],
│           
│                   [[ 1.4557, -0.8750,  0.9974],
│                    [-0.4267, -2.5494, -0.8116]],
│           
│                   [[ 1.4557, -0.8750,  0.9974],
│                    [-0.4267, -2.5494, -0.8116]],
│           
│                   [[ 1.4557, -0.8750,  0.9974],
│                    [-0.4267, -2.5494, -0.8116]],
│           
│                   [[ 1.4557, -0.8750,  0.9974],
│                    [-0.4267, -2.5494, -0.8116]],
│           
│                   [[ 1.4557, -0.8750,  0.9974],
│                    [-0.4267, -2.5494, -0.8116]],
│           
│                   [[ 1.4557, -0.8750,  0.9974],
│                    [-0.4267, -2.5494, -0.8116]]])
└── 'x' --> <FastTreeValue 0x7fc894edd3d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.2 µs ± 36.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc894de8160>
├── 'a' --> tensor([[ 1.4557, -0.8750,  0.9974],
│                   [-0.4267, -2.5494, -0.8116],
│                   [ 1.4557, -0.8750,  0.9974],
│                   [-0.4267, -2.5494, -0.8116],
│                   [ 1.4557, -0.8750,  0.9974],
│                   [-0.4267, -2.5494, -0.8116],
│                   [ 1.4557, -0.8750,  0.9974],
│                   [-0.4267, -2.5494, -0.8116],
│                   [ 1.4557, -0.8750,  0.9974],
│                   [-0.4267, -2.5494, -0.8116],
│                   [ 1.4557, -0.8750,  0.9974],
│                   [-0.4267, -2.5494, -0.8116],
│                   [ 1.4557, -0.8750,  0.9974],
│                   [-0.4267, -2.5494, -0.8116],
│                   [ 1.4557, -0.8750,  0.9974],
│                   [-0.4267, -2.5494, -0.8116]])
└── 'x' --> <FastTreeValue 0x7fc89ba63370>
    └── 'c' --> tensor([[ 1.5550,  1.7717,  0.8875, -0.2871],
                        [-0.0854, -0.6550,  0.6069, -0.3160],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 23.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.3 µs ± 79.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.5550,  1.7717,  0.8875, -0.2871],
                       [-0.0854, -0.6550,  0.6069, -0.3160],
                       [-0.1392,  1.0513,  1.9036, -1.3792]],
              
                      [[ 1.5550,  1.7717,  0.8875, -0.2871],
                       [-0.0854, -0.6550,  0.6069, -0.3160],
                       [-0.1392,  1.0513,  1.9036, -1.3792]],
              
                      [[ 1.5550,  1.7717,  0.8875, -0.2871],
                       [-0.0854, -0.6550,  0.6069, -0.3160],
                       [-0.1392,  1.0513,  1.9036, -1.3792]],
              
                      [[ 1.5550,  1.7717,  0.8875, -0.2871],
                       [-0.0854, -0.6550,  0.6069, -0.3160],
                       [-0.1392,  1.0513,  1.9036, -1.3792]],
              
                      [[ 1.5550,  1.7717,  0.8875, -0.2871],
                       [-0.0854, -0.6550,  0.6069, -0.3160],
                       [-0.1392,  1.0513,  1.9036, -1.3792]],

In [26]:
%timeit Batch.stack(batches)

78 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.5550,  1.7717,  0.8875, -0.2871],
                      [-0.0854, -0.6550,  0.6069, -0.3160],
                      [-0.1392,  1.0513,  1.9036, -1.3792],
                      [ 1.5550,  1.7717,  0.8875, -0.2871],
                      [-0.0854, -0.6550,  0.6069, -0.3160],
                      [-0.1392,  1.0513,  1.9036, -1.3792],
                      [ 1.5550,  1.7717,  0.8875, -0.2871],
                      [-0.0854, -0.6550,  0.6069, -0.3160],
                      [-0.1392,  1.0513,  1.9036, -1.3792],
                      [ 1.5550,  1.7717,  0.8875, -0.2871],
                      [-0.0854, -0.6550,  0.6069, -0.3160],
                      [-0.1392,  1.0513,  1.9036, -1.3792],
                      [ 1.5550,  1.7717,  0.8875, -0.2871],
                      [-0.0854, -0.6550,  0.6069, -0.3160],
                      [-0.1392,  1.0513,  1.9036, -1.3792],
                      [ 1.5550,  1.7717,  0.8875, -0.2871],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

337 µs ± 3.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
